In [3]:
# Configuración inicial
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Image
import seaborn as sns

# Configurar matplotlib para notebook
%matplotlib inline
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Librerías básicas cargadas")


ModuleNotFoundError: No module named 'seaborn'

In [1]:
# Importar nuestro sistema avanzado
try:
    # Agregar ruta del proyecto
    if '.' not in sys.path:
        sys.path.append('.')
    
    from src.agents.advanced_trading_analytics import AdvancedTradingAnalytics, Trade, PIDController
    print("✅ Sistema avanzado de trading importado")
    
    # Verificar dependencias opcionales
    try:
        import yfinance as yf
        print("✅ Yahoo Finance disponible")
    except ImportError:
        print("⚠️ Yahoo Finance no disponible - se usarán datos simulados")
    
    try:
        import ccxt
        print("✅ CCXT disponible para Binance")
    except ImportError:
        print("⚠️ CCXT no disponible - instalar: pip install ccxt")

except ImportError as e:
    print(f"❌ Error importando sistema: {e}")
    print("💡 Asegúrate de ejecutar desde la raíz del proyecto")


NameError: name 'sys' is not defined

In [ ]:
# Configuración del sistema
SYMBOL = 'SPY'  # S&P 500 ETF
USE_BINANCE = False  # Cambiar a True para usar Binance
DATA_PERIOD = '1y'  # Período de datos

print(f"🎯 Configuración:")
print(f"   Símbolo: {SYMBOL}")
print(f"   Fuente: {'Binance' if USE_BINANCE else 'Yahoo Finance (SP500)'}")
print(f"   Período: {DATA_PERIOD}")

# Crear sistema
system = AdvancedTradingAnalytics(symbol=SYMBOL, use_binance=USE_BINANCE)
print("\\n✅ Sistema inicializado")


In [ ]:
# Cargar datos
if USE_BINANCE:
    success = system.load_binance_data(timeframe='1h', limit=1000)
else:
    success = system.load_sp500_data(period=DATA_PERIOD)

if success:
    print(f"✅ Datos cargados exitosamente")
    print(f"📈 Total de puntos: {len(system.data)}")
    
    # Mostrar información de los datos
    data_info = pd.DataFrame({
        'Columna': system.data.columns,
        'Datos Disponibles': [system.data[col].notna().sum() for col in system.data.columns],
        'Datos Faltantes': [system.data[col].isna().sum() for col in system.data.columns]
    })
    
    display(HTML("<h4>📋 Información de los Datos:</h4>"))
    display(data_info)
    
    # Estadísticas básicas del precio
    price_stats = system.data['price'].describe()
    display(HTML("<h4>📊 Estadísticas del Precio:</h4>"))
    display(price_stats)
    
else:
    print("❌ Error cargando datos")


In [ ]:
# Ejecutar backtest avanzado
if success:
    print("🚀 Iniciando backtest avanzado...")
    print("⏳ Esto puede tomar unos momentos...")
    
    # Configurar parámetros del backtest
    start_step = 100  # Dar tiempo para que se calculen los indicadores
    end_step = len(system.data) - 50  # Dejar margen al final
    
    print(f"📊 Rango de backtest: {start_step} - {end_step} ({end_step - start_step} steps)")
    
    # Ejecutar
    metrics = system.run_backtest(start_step=start_step, end_step=end_step)
    
    if metrics and metrics.get('total_trades', 0) > 0:
        print("\\n🎉 ¡Backtest completado exitosamente!")
        
        # Mostrar resumen rápido
        quick_summary = f"""
        📈 RESUMEN RÁPIDO:
        • Total Trades: {metrics['total_trades']}
        • Win Rate: {metrics['win_rate']:.1f}%
        • Retorno Total: ${metrics['total_return_abs']:.2f}
        • Sharpe Ratio: {metrics['sharpe_ratio']:.3f}
        • Max Drawdown: ${metrics['max_drawdown']:.2f}
        """
        print(quick_summary)
        
    else:
        print("❌ No se generaron trades suficientes")
        print("💡 Intenta ajustar los parámetros o el período de datos")
else:
    print("⏭️ Saltando backtest por error en carga de datos")


In [ ]:
# Generar dashboard completo y visualizaciones separadas
if 'metrics' in locals() and metrics and success:
    print("🎨 Generando visualizaciones...")
    
    # Generar dashboard original
    system.create_comprehensive_dashboard()
    
    # Mostrar dashboard si existe (nueva ruta)
    dashboard_path = os.path.join('..', 'data', 'results', 'trading_analysis', 'visualizations', 'dashboard_avanzado.png')
    if os.path.exists(dashboard_path):
        display(HTML("<h3>📊 Dashboard Completo:</h3>"))
        display(Image(dashboard_path, width=1000))
        print(f"✅ Dashboard disponible en: {dashboard_path}")
    else:
        print("⚠️ Dashboard no encontrado en la nueva ubicación")
    
    print("✅ Dashboard completo generado")
    
else:
    print("❌ No hay datos para generar visualizaciones")


In [ ]:
# Ejecutar script para generar múltiples visualizaciones y CSV
import subprocess

print("🚀 Generando visualizaciones separadas y exportando CSV...")
print("⏳ Esto puede tomar unos momentos...")

try:
    # Ejecutar script de visualizaciones limpias (nueva ruta)
    script_path = os.path.join('..', 'scripts', 'generate_clean_visualizations.py')
    result = subprocess.run(['python', script_path], 
                          capture_output=True, text=True, timeout=120)
    
    if result.returncode == 0:
        print("✅ Visualizaciones generadas exitosamente")
        if result.stdout:
            print("📋 Salida del script:")
            print(result.stdout)
    else:
        print("⚠️ Script completado con advertencias")
        if result.stderr:
            print("Errores:", result.stderr)
            
except subprocess.TimeoutExpired:
    print("⏰ Script tardó demasiado, pero puede haber generado archivos")
except Exception as e:
    print(f"❌ Error ejecutando script: {e}")

# Verificar archivos generados en las nuevas ubicaciones
import glob

# Rutas de las nuevas ubicaciones
viz_dir = os.path.join('..', 'data', 'results', 'trading_analysis', 'visualizations')
csv_dir = os.path.join('..', 'data', 'results', 'trading_analysis', 'csv_exports')

png_files = sorted(glob.glob(os.path.join(viz_dir, "*.png")))
csv_files = sorted(glob.glob(os.path.join(csv_dir, "*.csv")))

print(f"\\n📁 Archivos PNG generados ({len(png_files)}):")
for file in png_files:
    size_mb = os.path.getsize(file) / (1024*1024)
    filename = os.path.basename(file)
    print(f"   • {filename} ({size_mb:.1f} MB)")

print(f"\\n📊 Archivos CSV generados ({len(csv_files)}):")
for file in csv_files:
    size_kb = os.path.getsize(file) / 1024
    filename = os.path.basename(file)
    print(f"   • {filename} ({size_kb:.1f} KB)")


In [ ]:
# Examinar los archivos CSV generados
csv_dir = os.path.join('..', 'data', 'results', 'trading_analysis', 'csv_exports')
csv_files_to_examine = [
    'trades_detallados.csv',
    'metricas_resumen.csv', 
    'estadisticas_adicionales.csv'
]

for csv_filename in csv_files_to_examine:
    csv_file = os.path.join(csv_dir, csv_filename)
    if os.path.exists(csv_file):
        display(HTML(f"<h4>📋 {csv_filename.replace('_', ' ').title()}:</h4>"))
        
        try:
            df = pd.read_csv(csv_file, encoding='utf-8-sig')
            display(df)
            
            # Información adicional para trades detallados
            if 'trades_detallados' in csv_filename:
                display(HTML(f"<p><strong>Total de registros:</strong> {len(df)}</p>"))
                if 'Ganancia_Perdida' in df.columns:
                    ganancia_count = (df['Ganancia_Perdida'] == 'GANANCIA').sum()
                    perdida_count = (df['Ganancia_Perdida'] == 'PERDIDA').sum()
                    display(HTML(f"<p><strong>Ganancias:</strong> {ganancia_count} | <strong>Pérdidas:</strong> {perdida_count}</p>"))
            
        except Exception as e:
            display(HTML(f"<p style='color: red;'>Error leyendo {csv_filename}: {e}</p>"))
        
        display(HTML("<hr>"))
    else:
        display(HTML(f"<p style='color: orange;'>⚠️ Archivo {csv_filename} no encontrado en {csv_dir}</p>"))


In [ ]:
# Mostrar las visualizaciones individuales generadas
viz_dir = os.path.join('..', 'data', 'results', 'trading_analysis', 'visualizations')
visualization_files = [
    ('01_precio_y_trades_detallado.png', 'Precio y Trades Detallado'),
    ('02_metricas_financieras_panel.png', 'Panel de Métricas Financieras'),
    ('03_distribuciones_estadisticas.png', 'Distribuciones y Estadísticas'),
    ('04_equity_y_drawdown.png', 'Equity y Drawdown'),
    ('05_analisis_performance.png', 'Análisis de Performance'),
    ('06_tabla_trades_detallada.png', 'Tabla de Trades Detallada')
]

for filename, title in visualization_files:
    full_path = os.path.join(viz_dir, filename)
    if os.path.exists(full_path):
        display(HTML(f"<h4>📊 {title}:</h4>"))
        display(Image(full_path, width=900))
        display(HTML("<hr>"))
    else:
        display(HTML(f"<p style='color: orange;'>⚠️ {filename} no encontrado en {viz_dir}</p>"))


In [ ]:
# Resumen final del análisis
if 'metrics' in locals() and metrics:
    
    final_summary = f"""
    🎉 ANÁLISIS COMPLETADO EXITOSAMENTE
    
    📊 RESULTADOS PRINCIPALES:
    • Sistema ejecutado con {metrics['total_trades']} trades
    • Win Rate alcanzado: {metrics['win_rate']:.1f}%
    • Retorno total generado: ${metrics['total_return_abs']:.2f}
    • Sharpe Ratio: {metrics['sharpe_ratio']:.3f}
    • Control PID y MAPE implementados
    
    📁 ARCHIVOS GENERADOS:
    • {len([f for f in glob.glob('*.png')])} visualizaciones PNG
    • 3 archivos CSV con datos detallados
    • 1 notebook Jupyter ejecutable
    
    🔧 CARACTERÍSTICAS IMPLEMENTADAS:
    ✅ Tracking completo de trades con IDs únicos
    ✅ Métricas financieras avanzadas (Sharpe, Drawdown, Profit Factor)
    ✅ Control PID para optimización de señales
    ✅ MAPE para evaluación de predicciones ML
    ✅ Exportación completa a CSV
    ✅ Visualizaciones separadas y claras
    ✅ Integración con datos SP500 (Yahoo Finance)
    ✅ Preparación para trading Bitcoin (Binance)
    """
    
    print(final_summary)
    
    # Recomendaciones
    display(HTML("""
    <h3>🚀 PRÓXIMOS PASOS RECOMENDADOS:</h3>
    <ol>
        <li><strong>Optimización de Parámetros:</strong> Experimenta con diferentes valores de PID (Kp, Ki, Kd)</li>
        <li><strong>Backtesting Extendido:</strong> Prueba con períodos más largos (2-5 años)</li>
        <li><strong>Trading en Vivo:</strong> Configura Binance API para Bitcoin trading</li>
        <li><strong>Machine Learning:</strong> Entrena modelos con más datos históricos</li>
        <li><strong>Risk Management:</strong> Ajusta stop-loss y take-profit según volatilidad</li>
    </ol>
    
    <h3>📋 ARCHIVOS PARA USAR:</h3>
    <ul>
        <li><strong>trades_detallados.csv:</strong> Todos los trades con información completa</li>
        <li><strong>metricas_resumen.csv:</strong> KPIs del sistema</li>
        <li><strong>estadisticas_adicionales.csv:</strong> Métricas adicionales</li>
        <li><strong>advanced_trading_notebook.ipynb:</strong> Este notebook para análisis futuro</li>
        <li><strong>Visualizaciones PNG:</strong> Para presentaciones e informes</li>
    </ul>
    """))
    
else:
    print("❌ No hay métricas disponibles para el resumen")
